# Deteksi Wajah

In [1]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [2]:
import cv2

# Membaca cascade classifier untuk deteksi wajah
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Membuka kamera laptop
cap = cv2.VideoCapture(1)

while True:
    # Membaca frame dari kamera
    ret, frame = cap.read()

    # Mengubah frame ke dalam grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Mendeteksi wajah dalam frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Menandai wajah yang terdeteksi dengan kotak
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

    # Menampilkan frame hasil deteksi
    cv2.imshow('Object Detection', frame)

    # Tombol 'q' untuk keluar dari loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Melepaskan sumber video dan menutup semua jendela
cap.release()
cv2.destroyAllWindows()

# Memberi Label Pada Wajah Yang Terdeteksi

In [23]:

import cv2

# Membaca cascade classifier untuk deteksi wajah
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Membuka kamera laptop
cap = cv2.VideoCapture(1)

# Label kepemilikan
owner_label = "Agfan"

while True:
    # Membaca frame dari kamera
    ret, frame = cap.read()

    # Mengubah frame ke dalam grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Mendeteksi wajah dalam frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Menandai wajah yang terdeteksi dengan kotak dan menambahkan label kepemilikan
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(frame, owner_label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Menampilkan frame hasil deteksi
    cv2.imshow('Object Detection', frame)

    # Tombol 'q' untuk keluar dari loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Melepaskan sumber video dan menutup semua jendela
cap.release()
cv2.destroyAllWindows()


# Menyimpan gambar wajah yang terdeteksi

In [17]:
import cv2
import os

# Membuat direktori untuk menyimpan gambar yang terdeteksi
output_folder = "detected_faces"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Membuka kamera laptop
cap = cv2.VideoCapture(1)

# Membaca cascade classifier untuk deteksi wajah
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Label kepemilikan
owner_label = "Agfan"

# Counter untuk nama file
file_counter = 0

while True:
    # Membaca frame dari kamera
    ret, frame = cap.read()

    # Mengubah frame ke dalam grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Mendeteksi wajah dalam frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Menandai wajah yang terdeteksi dengan kotak dan menambahkan label kepemilikan
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(frame, owner_label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Menampilkan frame hasil deteksi
    cv2.imshow('Object Detection', frame)

    # Menyimpan gambar yang terdeteksi ketika tombol 's' ditekan
    if cv2.waitKey(1) & 0xFF == ord('s'):
        # Menyimpan gambar dengan nama unik di dalam folder output_folder
        for i, (x, y, w, h) in enumerate(faces):
            face_img = frame[y:y+h, x:x+w]
            img_name = os.path.join(output_folder, f"{owner_label}_{file_counter}.jpg")
            while os.path.exists(img_name): # Mengecek apakah nama file sudah digunakan
                file_counter += 1
                img_name = os.path.join(output_folder, f"{owner_label}_{file_counter}.jpg")
            cv2.imwrite(img_name, face_img)
            file_counter += 1
        print(f"{len(faces)} detected face(s) saved.")

    # Tombol 'q' untuk keluar dari loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Melepaskan sumber video dan menutup semua jendela
cap.release()
cv2.destroyAllWindows()


# Membuat dataset dan melakukan ekstrasi feature HOG

In [11]:
import cv2
import csv
import os
from skimage.feature import hog

# Fungsi untuk mengekstrak fitur HOG dari gambar wajah
def extract_hog_features(image_path):
    # Memuat gambar
    image = cv2.imread(image_path)
    # Mengubah gambar ke dalam grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Menerapkan HOG
    features = hog(gray, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), block_norm='L2-Hys')
    # Mengembalikan fitur HOG
    return features

# Direktori folder yang berisi gambar wajah
folder_path = "detected_faces"

# Inisialisasi dataset
dataset = []

# Iterasi melalui file-file dalam folder
for filename in os.listdir(folder_path):
    if filename.endswith(".jpg") or filename.endswith(".jpeg") or filename.endswith(".png"):
        # Ambil nama label dari nama file (tanpa ekstensi dan angka setelah underscore)
        label = os.path.splitext(filename.split('_')[0])[0]
        # Buat path lengkap ke gambar
        image_path = os.path.join(folder_path, filename)
        # Ekstrak fitur-fitur HOG dari gambar wajah
        features = extract_hog_features(image_path)
        # Konversi fitur HOG menjadi string
        features_str = ' '.join(map(str, features))
        # Tambahkan informasi gambar dan fitur-fiturnya ke dataset
        dataset.append({"image_path": image_path, "label": label, "hog_features": features_str})

# Simpan dataset ke dalam file CSV
with open('dataset.csv', mode='w', newline='') as file:
    fieldnames = ['image_path', 'label', 'hog_features']
    writer = csv.DictWriter(file, fieldnames=fieldnames)

    writer.writeheader()
    for data in dataset:
        writer.writerow(data)


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'detected_faces'

# Mendeteksi Wajah dari Dataset yang diberikan

In [12]:
import csv
import cv2
import numpy as np
from skimage.feature import hog

# Fungsi untuk mengekstrak fitur HOG dari gambar wajah
def extract_hog_features(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    features = hog(gray, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), block_norm='L2-Hys')
    return features

# Fungsi untuk memuat dataset dari file CSV
def load_dataset(csv_file):
    dataset = []
    # Atur ukuran batas bidang agar lebih besar
    csv.field_size_limit(1000000)
    with open(csv_file, mode='r', newline='', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        for row in reader:
            dataset.append(row)
    return dataset

# Fungsi untuk menormalkan panjang vektor fitur HOG
def normalize_hog_features(features, length):
    if len(features) < length:
        features = np.pad(features, (0, length - len(features)), mode='constant')
    elif len(features) > length:
        features = features[:length]
    return features

# Fungsi untuk membandingkan fitur HOG dari wajah yang terdeteksi dengan dataset
def match_face(features, dataset):
    best_match = None
    best_score = float('inf')
    for data in dataset:
        dataset_features = np.array(data['hog_features'].split(), dtype=float)
        dataset_features = normalize_hog_features(dataset_features, len(features))
        score = np.linalg.norm(features - dataset_features)
        if score < best_score:
            best_score = score
            best_match = data['label']
    return best_match

# Memuat dataset
dataset = load_dataset('dataset.csv')

# Membaca cascade classifier untuk deteksi wajah
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Membuka kamera
cap = cv2.VideoCapture(1)

while True:
    # Membaca frame dari kamera
    ret, frame = cap.read()

    # Deteksi wajah dalam frame
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Loop melalui setiap wajah yang terdeteksi
    for (x, y, w, h) in faces:
        face_image = frame[y:y+h, x:x+w]  # Mendapatkan gambar wajah
        face_features = extract_hog_features(face_image)  # Ekstrak fitur HOG dari wajah
        face_label = match_face(face_features, dataset)  # Mencocokkan wajah dengan dataset
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)  # Gambar kotak di sekitar wajah
        cv2.putText(frame, face_label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)  # Menambahkan label di atas wajah

    # Menampilkan frame dengan wajah yang terdeteksi
    cv2.imshow('Face Detection', frame)

    # Tombol 'q' untuk keluar dari loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Melepaskan sumber video dan menutup semua jendela
cap.release()
cv2.destroyAllWindows()


FileNotFoundError: [Errno 2] No such file or directory: 'dataset.csv'

In [9]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense

# Langkah 1: Pemrosesan Gambar
image_size = (224, 224)
batch_size = 32

# Langkah 2: Pemisahan Data
train_data_dir = 'detected_faces'
validation_data_dir = 'validasi'

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical')

validation_datagen = ImageDataGenerator(rescale=1./255)

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical')

# Langkah 3: Pembuatan Model Ekstraksi Fitur
base_model = MobileNetV2(weights='imagenet', include_top=False)

x = base_model.output
x = GlobalAveragePooling2D()(x)
output = Dense(train_generator.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

# Langkah 4: Pelatihan Model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

epochs = 10

model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size)

# Langkah 5: Evaluasi Model
# Evaluasi model sesuai kebutuhan Anda, misalnya menggunakan metrik akurasi atau F1-score.

# Langkah 6: Penggunaan Model
# Setelah model dilatih, Anda dapat menggunakannya untuk mengekstraksi fitur dari gambar baru.


TypeError: Descriptors cannot be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates

In [21]:
import cv2

def check_available_cameras():
    available_cameras = []
    index = 0
    while True:
        cap = cv2.VideoCapture(index)
        if not cap.read()[0]:
            break
        else:
            available_cameras.append(index)
            cap.release()
        index += 1
    return available_cameras

available_cameras = check_available_cameras()
print("Available cameras:", available_cameras)


Available cameras: [0, 1, 2, 3]
